#BERT & SBERT Fine-Tuning

I shall retrain a BERT model based on MLM objectve to be better accustomed to the vocabulry at hand. To obtain good single vector representations, an SBERT model shall be trained based on this BERT model.



In [ ]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=324a74a79d37ff60d239df850edfeecf30d4f7abb146cd7883497f9b7d56789a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd

## Fine-tuning the BERT model for MLM

https://huggingface.co/learn/nlp-course/chapter7/3?fw=tf
https://www.analyticsvidhya.com/blog/2022/09/fine-tuning-bert-with-masked-language-modeling/


In [ ]:
import transformers

In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFDistilBertForMaskedLM.

All the weights of TFDistilBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
from transformers import BertTokenizer, TFBertForMaskedLM
import tensorflow as tf
import os
import numpy as np
import re
import matplotlib.pyplot as plt
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [1]:
# connecting to drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [7]:
## reading in tabuar data

import pandas as pd
airbnb_london_filtered_images = pd.read_csv("/content/gdrive/My Drive/Thesis/London_Data/discriminator_tabular_data_images.csv")

In [8]:
airbnb_london_filtered_images[airbnb_london_filtered_images.neighborhood_overview  == "No description"].shape[0] / airbnb_london_filtered_images.shape[0]


0.3025254762959681

In cases where ne space/neighborhood description is present, I shall replace the nan with a simple empty string.

In [ ]:

airbnb_london_filtered_images.neighborhood_overview = airbnb_london_filtered_images.neighborhood_overview.replace("No description", "")
airbnb_london_filtered_images["joint_description"] = airbnb_london_filtered_images.description + " . " + airbnb_london_filtered_images.neighborhood_overview


Only sentences to be used as input in MLM modelling.

In [ ]:
text_lst = []
len_lst = []
for text in airbnb_london_filtered_images["joint_description"]:
  splitted_text = text.split('.')
  for line in splitted_text:
      text_lst.append(line)
      len_lst.append(len(line.split(" ")))

In [ ]:
inputs = tokenizer(text_lst,max_length=100,truncation=True,padding='max_length',return_tensors='tf') ## using rather short max length here


In [ ]:
inputs['labels'] = inputs['input_ids']

In [ ]:
### performing the masking:

inp_ids = []
lbs = []
idx = 0
for inp in inputs.input_ids.numpy():
    actual_tokens = list(set(range(100)) -
                         set(np.where((inp == 101) | (inp == 102)
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens),
                                     size=num_of_token_to_mask,
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
inp_ids = tf.convert_to_tensor(inp_ids)
inputs['input_ids'] = inp_ids

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [ ]:
## actually training the model:

# on A100 GPU, one iteration takes roughly 40-45 minutes, thus 10 iters take around 7hrs!

history = model.fit([inputs.input_ids,inputs.attention_mask],inputs.labels,verbose=1,batch_size=8,epochs=10)


This modl (.bin and config data) shall be pushed to a HN repo to be called seamlessly as a models.Transformer() object (see below.)
Thus, they shall be saved locally, downloaded and manually uplooaded to an HF repo. There were some issues with the 'push_to_repo' function.

In [ ]:
model.save_pretrained("Local_Nico97/Local_BERT_tuned_AirBnB/pytorch_model.pt")

# this creates config.json and pytorch_model.bin data

For ose the fine-tuned bert model as an HF model.transformer(), one also needs to add a tokenizer.json adn a vocab.txt file to the repo. This can be done by downloading these from the BERT base repo and uploading them to the repo under consideration.

## Now: Fine-tuning a Sentence Transformer model

The tuned BERT model (above) is now integrated into an SBERT model.
The fine-tuning if such a model is possible as the similarity of the names and the descriptions can be used as training data.

In [ ]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sentence_transformers import SentenceTransformer, models
import torch

##

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
word_embedding_model = models.Transformer("Nico97/BERT_tuned_AirBnB")

In [ ]:
word_embedding_model.get_word_embedding_dimension()

768

In [ ]:
## Loading in BERT and setting up an SBERT model


## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument -
model_sbert = SentenceTransformer(modules=[word_embedding_model, pooling_model])

### Preparing the data in the correct, needed format

Format of list containing both strings (at index positions 0 and 1) per datapoint is needed!

In [ ]:
airbnb_london_filtered_images.index = list(range(airbnb_london_filtered_images.shape[0]))

In [ ]:
train_examplesB = []

for i in range(airbnb_london_filtered_images.shape[0]):
  example = airbnb_london_filtered_images.iloc[i]
  train_examplesB.append(InputExample(texts=[ airbnb_london_filtered_images.name[i], airbnb_london_filtered_images.joint_description[i] ] ))

Using the MN Ranking Loss as the data is in the format of two related strings.

https://www.sbert.net/docs/package_reference/losses.html

In [ ]:
train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=32)
train_lossB = losses.MultipleNegativesRankingLoss(model=model_sbert)
num_epochsB = 10
warmup_stepsB = int(len(train_dataloaderB) * num_epochsB * 0.1) #10% of train data

In [ ]:
## run on A100 GPU!

model_sbert.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1522 [00:00<?, ?it/s]

In [ ]:
import torch

#saving the model
model_save_name = 'sbert_tuned.pth'

path ="/content/gdrive/My Drive/Thesis/Models/{}".format(model_save_name)
torch.save(model_sbert, path)
